# 🔍 Fun with Keyword Search using VideoDB

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Keyword_Search_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 💬 Overview
---


In this tutorial, let’s explore the powerful functionality of Keyword Search in VideoDB. This feature enables users to efficiently locate any keyword or phrase within their video assets, streamlining the process of content discovery.

![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Keyword_Search_1.png)

![](https://raw.githubusercontent.com/video-db/videodb-cookbook/main/images/Intro_Outro_Inline/image.png)

## Setup
---

### 📦  Installing packages

In [ ]:
%pip install -q videodb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [ ]:
import videodb
import os
from getpass import getpass

# Prompt user for API key securely
api_key = getpass("Please enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

Please enter your VideoDB API Key: ··········


## Steps
---

### 🌐 Step 1: Connect to VideoDB
Begin by establishing a connection to VideoDB using your API key

In [ ]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Videos

Upload the video to your VideoDB collection. You can upload the video asset from your local device or from a YouTube URL to upload the video from its source. This works as the base video for all the Keyword Search queries.

In [ ]:
video = coll.upload(url="https://www.youtube.com/watch?v=Uvufun6xer8")
print("Video ID: ", video.id)

Video ID:  m-z-019b9230-6a7f-7eb0-814a-a1fb1b4d1a43


### Lets play the original video

In [ ]:
video.play()

>You can upload from your local file system too by passing `file_path` in `upload()`

### 🔊  Step 3: Index Spoken Words

Index the spoken words in your video to enable  keyword search.


In [ ]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:15<00:00,  1.33it/s]


### 🔎  Step 4: Search for any keyword

Utilize the keyword search by using `Video.search()` method with following parameters.

* pass search query in `query` parameter
* pass `SearchType.keyword` in `search_type`

In [ ]:
from videodb import SearchType

results = video.search(query='metaverse', search_type=SearchType.keyword)

results.get_shots()

[Shot(video_id=m-z-019b9230-6a7f-7eb0-814a-a1fb1b4d1a43, video_title=The Metaverse and How We'll Build It Together -- Connect 2021, start=23.44, end=24.08, text=Metaverse, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9230-6a7f-7eb0-814a-a1fb1b4d1a43, video_title=The Metaverse and How We'll Build It Together -- Connect 2021, start=78.19, end=78.83, text=Metaverse, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9230-6a7f-7eb0-814a-a1fb1b4d1a43, video_title=The Metaverse and How We'll Build It Together -- Connect 2021, start=108.73, end=109.33, text=Metaverse, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9230-6a7f-7eb0-814a-a1fb1b4d1a43, video_title=The Metaverse and How We'll Build It Together -- Connect 2021, start=143.83, end=144.43, text=Metaverse, search_score=1, stream_url=None, player_url=None),
 Shot(video_id=m-z-019b9230-6a7f-7eb0-814a-a1fb1b4d1a43, video_title=The Metaverse and How We'll Build I

## 🔎 Refining Keyword Search results by adding padding.

Some keyword search results/ compilations may appear slightly choppy, or the cuts may feel abrupt. We can solve this issue by using VideoDB’s padding controls. Here’s how it works:


![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Keyword_Search_2.png)

The resulting shots can be made smoother by including a little more context from before and after the matching timestamps. That’s exactly what padding controls enable:

Using the Editor SDK's `Track` and `Clip` pattern, we can create a timeline with padding:

1. Create a timeline and track using `Timeline()` and `Track()`
2. Create a `VideoAsset` with `id` and `start` parameters (where `start` is adjusted by subtracting padding)
3. Wrap each asset in a `Clip` with the appropriate duration (adding padding on both ends)
4. Add clips to the track sequentially using `track.add_clip(start_time, clip)`

In [ ]:
from videodb import play_stream
from videodb.editor import Timeline, Track, Clip, VideoAsset

timeline = Timeline(conn)

# Add padding for smoother cuts
padding = 0.4

# Create main track
track = Track()
seeker = 0

# Compile Video from search results
for shot in results.shots:
    start_with_padding = max(0, shot.start - padding)
    duration = (shot.end + padding) - start_with_padding

    asset = VideoAsset(id=shot.video_id, start=start_with_padding)
    clip = Clip(asset=asset, duration=duration)
    track.add_clip(seeker, clip)

    seeker += duration

timeline.add_track(track)

stream_url = timeline.generate_stream()
play_stream(stream_url)

## 🎉 Conclusion
---
Keyword Search in VideoDB empowers users to extract valuable insights from their video assets with ease. For more information and advanced features, explore the [VideoDB Documentation](https://docs.videodb.io/) and join the VideoDB community on [GitHub](https://github.com/video-db) or [Discord](https://discord.com/invite/py9P639jGz) for support and collaboration.
